In [68]:
import numpy as np

## Deriving P with method 4.1 from the paper

Consider a coordinate system centered in the center of the table, with $x$ and $y$ directions parallel to its edges. Once we have $K$, $R$ can be computed by considering that the rays through the projections of the vanishing points must be perpendicular.

Let $\mathbf u$ and $\mathbf v$ be the projections vanishing points on the plane $z=k$ in Euclidean coordinates (these can be obtained by inverting $K$ on the pixel coordinates). We know $\mathbf u \cdot \mathbf v = 0$, so solving for $k$ we get $k^2 = -x_u x_v - y_u y_v$. Immediately we get $\mathbf x' = \frac{\mathbf u}{||\mathbf u||}$ and $\mathbf y' = \frac {\mathbf v} {||\mathbf v||}$, and of course $\mathbf z' = \mathbf x' \times \mathbf y'$. Lastly,

$$
R = \begin{pmatrix} 
\\
\mathbf x' & \mathbf y' & \mathbf z' \\
\\
\end{pmatrix}
$$.

Once the intrinsics $\alpha$, $u_0$, $v_0$ and $f$ are known, the unit vector $\hat{\mathbf q}_a$ corresponding to the direction of the ray projected on a generic point $A$ on the picture can be computed. Let $u$ and $v$ be the (normalized) pixel coordinates of $A$ on the picture. By equation (2) on the paper, its projection on the image plane will be the vector $\mathbf q_a = \begin{pmatrix}\alpha_u (u-u_0) & k & \alpha_v(v_0 - v)\end{pmatrix}$. $\alpha_u$, $\alpha_v$ and $k$ are not known, but by dividing the vector by $\alpha_v$ a new vector on the same ray is obtained, whose coordinates are:

$$
\frac{\mathbf q_a}{\alpha_v} =
\begin{pmatrix}\frac{\alpha_u}{\alpha_v}(u-u_0) & \frac{k}{\alpha_v} & (v_0 - v)\end{pmatrix} =
\begin{pmatrix}\frac{1}{\alpha}(u-u_0) & f & (v_0 - v)\end{pmatrix}
$$

The unit vector $\hat{\mathbf q}_a$ corresponding to the ray can be obtained simply by normalizing the latter.

In [80]:
# All arguments are 3D vectors
def corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qa_norm):
    return np.dot(np.cross(qn_norm, qm_norm), qh_norm)/np.dot(np.cross(qn_norm, qm_norm), qa_norm) * qa_norm

# Normalized vector in camera coordinates corresponding to point in pixel coordinates
def qnorm(u_px, v_px, u_0, v_0, alpha, f):
    q = np.array([(u_px - u_0)/alpha, f, v_0 - v_px])
    return q/np.linalg.norm(q)

$\lambda_h$ can be derived imposing the area of the table surface as a constraint:

$$
AB \cdot AC = A \\
A = |\mathbf p_B - \mathbf p_A|\cdot|\mathbf p_C - \mathbf p_A| \\
A = |\lambda_h \mathbf c_B - \lambda_h \mathbf c_A|\cdot|\lambda_h \mathbf c_C - \lambda_h \mathbf c_A| \\
\lambda_h = \sqrt \frac{A}{|\mathbf c_B - \mathbf c_A|\cdot|\mathbf c_C - \mathbf c_A|}
$$

$\mathbf c_A$ is the vector obtaining in equation (15) of the paper by decomposing $\mathbf t_{wc}$ as in equation (13):

$$
\mathbf p_A =
\frac{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \mathbf t_{wc}}{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_a} \hat{\mathbf q}_a =
\frac{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \lambda_h \hat{\mathbf q}_h}{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_a} \hat{\mathbf q}_a =
\lambda_h \frac{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_h}{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_a} \hat{\mathbf q}_a
=: \lambda_h \mathbf c_A
$$

In [81]:
def hom2eucl(v):
        assert(v[-1] != 0), f"Point at infinity {v} does not have an Euclidean correspondent!"
        return v[:-1]/v[-1]

In [82]:
# Vertex A of the table must be consecutive to both vertices B and C;
# H is the center of the table.
# vpx, vpy, uv* must be given in Euclidean pixel coordinates.
def find_projection_matrix2(uv_princ, alpha, f, vpx, vpy, uva, uvb, uvc, uvh, table_area):
    def normalize(v):
        return v/np.linalg.norm(v)
    qm_norm = qnorm(*vpx, *uv_princ, alpha, f)
    qn_norm = qnorm(*vpy, *uv_princ, alpha, f)
    qz_norm = np.cross(qm_norm, qn_norm)
    R = np.column_stack((qm_norm, qn_norm, qz_norm))
    
    qh_norm = qnorm(*uvh, *uv_princ, alpha, f)
    qa_norm = qnorm(*uva, *uv_princ, alpha, f)
    qb_norm = qnorm(*uvb, *uv_princ, alpha, f)
    qc_norm = qnorm(*uvc, *uv_princ, alpha, f)
    
    ca = corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qa_norm)
    cb = corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qb_norm)
    cc = corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qc_norm)
    
    lambda_h = np.sqrt(table_area/(np.linalg.norm(cb - ca)*np.linalg.norm(cc - ca)))
    t = lambda_h * qh_norm
    
    Rt = np.column_stack((R, t))
    return Rt, ca*lambda_h, cb*lambda_h, cc*lambda_h

In [83]:
def norm_coordinates(v, normf):
    res = np.copy(v)
    res[0:2] = res[0:2]/normf
    return res

uv_princ, alpha, f = np.array([1.59572975, 1.15771016]), 1.0, 3.1290276507245047

with open("checkers_fullsize/outs.csv") as inf:
    inrows = np.array([l.split(",") for l in inf.read().splitlines()]).astype("float")
vpxs, vpys, As, Bs, Cs, centers = inrows.reshape((12, -1, 3)).swapaxes(0,1)
normf = 1000
img_idx = 2
vpx = hom2eucl(norm_coordinates(vpxs[img_idx], normf))
vpy = hom2eucl(norm_coordinates(vpys[img_idx], normf))
uva = hom2eucl(norm_coordinates(As[img_idx], normf))
uvb = hom2eucl(norm_coordinates(Bs[img_idx], normf))
uvc = hom2eucl(norm_coordinates(Cs[img_idx], normf))
uvh = hom2eucl(norm_coordinates(centers[img_idx], normf))
table_area = 0.06237 # square meters
Rt, pa, pb, pc = find_projection_matrix2(uv_princ, alpha, f, vpx, vpy, uva, uvb, uvc, uvh, table_area)

In [76]:
def angle_between(v1, v2):
    return np.rad2deg(np.arccos(np.dot(v1, v2)/np.linalg.norm(v1)*np.linalg.norm(v2)))

db, dc = pa-pb, pa-pc
np.linalg.norm(db), np.linalg.norm(dc), angle_between(db, dc), angle_between(np.transpose(Rt[:,1]), np.transpose(Rt[:,0]))

(0.21187011416780482,
 0.29437846977607185,
 89.97474854455092,
 89.70860896479203)

In [77]:
table_plane = np.append(Rt[:,2], -np.dot(Rt[:,2], Rt[:,3]))
table_plane

array([ 0.02331243, -0.88294455,  0.46887055,  0.4697845 ])

In [85]:
width_px = 3264
normf = 1000
fov_angle = np.rad2deg(2*np.arctan(width_px/(2*f*normf)))
fov_angle

55.090187916693004